# **Predict Future Sales**

# **Dataset Description**
You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

**File descriptions**

**sales_train.csv** - the training set. Daily historical data from January 2013 to October 2015.

**test.csv** - the test set. You need to forecast the sales for these shops and products for November 2015.

**sample_submission.csv** - a sample submission file in the correct format.

**items.csv** - supplemental information about the items/products.

**item_categories.csv ** - supplemental information about the items categories.

**shops.csv**- supplemental information about the shops.

**Data fields**
**ID** - an Id that represents a (Shop, Item) tuple within the test set

**shop_id** - unique identifier of a shop

**item_id** - unique identifier of a product

**item_category_id** - unique identifier of item category

**item_cnt_day** - number of products sold. You are predicting a monthly
amount of this measure

**item_price** - current price of an item

**date** - date in format dd/mm/yyyy

**date_block_num** - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33


**item_name **- name of item

**shop_name** - name of shop

**item_category_name** - name of item category

# **DQC **
Accuracy, completeness, consistency, validity, uniqueness, and timeliness.

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
import xgboost as xgb

from sklearn.preprocessing import MinMaxScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Analys of sales_train

In [4]:
sales_train = pd.read_csv("/content/drive/MyDrive/Predict Future Sales/sales_train.csv")
sales_train.info()
sales_train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 299.6 MB


,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [ ]:
#Расчет пропусков в процентах
pd.DataFrame(sales_train.isnull().sum()/len(sales_train)*100, columns = ['Количество пропусков (%)'])

# Преобразуем дату в нужный формат

In [5]:
sales_train['date'] = pd.to_datetime(sales_train['date'], format='%d.%m.%Y')

In [7]:
# Посмотрим на количество магазинов
# sales_train["shop_id"].value_counts()
len(sales_train["shop_id"].unique())

60

Посмотрим, сколько у нас магазинов находится в тестовом наборе и удалим те, которые отсутствуют в нем.

In [9]:
df_test = pd.read_csv("/content/drive/MyDrive/Predict Future Sales/test.csv")
shop_list = list(df_test.shop_id.unique())
# shop_list

In [10]:
print(len(sales_train))
sales_train = sales_train[sales_train.shop_id.isin(shop_list)]
print(len(sales_train))

2935849
2413246


In [11]:
len(sales_train["shop_id"].unique())

42

In [13]:
sales_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0
2935845,2015-10-09,33,25,7460,299.00,1.0
2935846,2015-10-14,33,25,7459,349.00,1.0
2935847,2015-10-22,33,25,7440,299.00,1.0


**Analys of sales_train**

In [22]:
df_items = pd.read_csv("/content/drive/MyDrive/Predict Future Sales/items.csv")
item_list = list(df_test.item_id.unique())
df_items.info()
df_items.describe()
df_items.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


Хороший вопрос - стоит ли удалять из объектов исследования те item,  которые отсутсвуют в тестовом наборе? По сути, они не нуждаются в предсказании.

In [15]:
print(len(sales_train))
sales_train = sales_train[sales_train.item_id.isin(item_list)]
print(len(sales_train))

2413246
1224439


**Analys of item_categories**




In [18]:
df_item_cat = pd.read_csv("/content/drive/MyDrive/Predict Future Sales/item_categories.csv")
df_item_cat.info()
df_item_cat.describe()
df_item_cat.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


# Посмотрим, сколько уникальных категорий в датасете и, в случае, если их очень много, попробуем агрегировать их в более курпные группы

In [19]:
len(list(df_item_cat.item_category_id.unique()))

84

Укрупним пока простой эвристикой, позже, если понадобится, сделаем более мелкое деление на группы

In [20]:
# Напишем функцию, которая будет получать первое слово из строки, для обозначения категории
def assign_cat(row):
    category = str(row).split()[1].strip()
    return category
df_item_cat['category'] = df_item_cat.apply(assign_cat, axis=1)
# Удалим название категории, т.к. их слишком много, мы будем использовать агрегированные данные
df_item_cat.drop('item_category_name',axis=1, inplace=True)
df_item_cat.head()

,item_category_id,category
0,0,PC
1,1,Аксессуары
2,2,Аксессуары
3,3,Аксессуары
4,4,Аксессуары


In [21]:
# Посмотрим, сколько получилось категорий
df_item_cat.category.value_counts()

Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Музыка         6
Программы      6
Карты          5
Кино           5
Служебные      2
Чистые         2
PC             1
Билеты         1
Доставка       1
Элементы       1
Name: category, dtype: int64

Объединим датасеты df_item_cat и df_items для дальнейшего слияния с общим тренировочным датасетом

In [23]:
df_items = df_items.merge(df_item_cat, on="item_category_id", how="left")
df_items.head()

,item_name,item_id,item_category_id,category
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,Программы
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино
4,***КОРОБКА (СТЕКЛО) D,4,40,Кино


Удалим названия товаров и item_category_id, чтоды облегчить дальнейшия операции слияния.

In [24]:
df_items = df_items.drop(labels = ['item_name','item_category_id'], axis = 1)
df_items.head()

,item_id,item_category_id,category
0,0,40,Кино
1,1,76,Программы
2,2,40,Кино
3,3,40,Кино
4,4,40,Кино


# **Analys of shops**

In [25]:
df_shops = pd.read_csv("/content/drive/MyDrive/Predict Future Sales/shops.csv")
df_shops.info()
df_shops.describe()
df_shops.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  60 non-null     object
 1   shop_id    60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


Попробуем по shop_name выделить новый признак - наименование города, которое потом можно с помощью One Hot incoding закадировать. Т.к. признак будет отражать размер города, в котором находится магазин, думаю, в будущем стоит обратиться за внешней информацией и внести дополнительный признак - численность города.
Сразу удалим shop_name.

In [26]:
# Напишем функцию, которая будет получать первое слово из строки, для обозначения города
def assign_city(row):
    city = str(row).split()[1].strip()
    return city
df_shops['City'] = df_shops.apply(assign_city, axis=1)
# Удалим лишние знаки, чтобы слить одинаковые города
df_shops['City'] = df_shops.City.apply(lambda x: x.replace("!",""))
df_shops.drop('shop_name',axis=1, inplace=True)
df_shops.head()

,shop_id,City
0,0,Якутск
1,1,Якутск
2,2,Адыгея
3,3,Балашиха
4,4,Волжский


In [27]:
# Посмотрим, сколько городов у нас получилось и нет ли дубликатов из-за орфографии
df_shops.City.value_counts()

Москва              13
Якутск               4
РостовНаДону         3
Воронеж              3
Тюмень               3
Новосибирск          2
Н.Новгород           2
Самара               2
Красноярск           2
Казань               2
Жуковский            2
Уфа                  2
СПб                  2
Томск                1
Сургут               1
Сергиев              1
Химки                1
Цифровой             1
Чехов                1
Мытищи               1
Омск                 1
Адыгея               1
Курск                1
Коломна              1
Калуга               1
Интернет-магазин     1
Выездная             1
Вологда              1
Волжский             1
Балашиха             1
Ярославль            1
Name: City, dtype: int64

Давайте обогатим наш датасет внешними данными и внесем численность населения в признаковое пространство



In [50]:
url = 'https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
df_population = pd.read_html(url)[0]
df_population = df_population[['Город', 'Население']]
df_population = df_population.rename(columns={'Город': 'City', 'Население': 'Population'})
df_population
df_population['Population'] = df_population.Population.apply(lambda x: x.replace("[2]",""))
# df_population['Population'] = df_population.Population.apply(lambda x: int(x.replace("[2]","")))

In [52]:
df_population['Population'] = df_population.Population.apply(lambda x: x.replace("[3]",""))
df_population['Population'] = df_population.Population.apply(lambda x: int(x.replace(" ","")))

In [53]:
# Заменим на правильные наименовани городов
# will remap the values
dict = {'СПб' : 'Санкт-Петербург', 'РостовНаДону' : 'Ростов-на-Дону', 'Н.Новгород' : 'Нижний Новгород'}
# Remap the values of the dataframe
df_shops = df_shops.replace({"City": dict})

In [54]:
def fill_population(df_shops, df_population):
    # Merge the two datasets on the 'City' column
    merged_df = df_shops.merge(df_population, on='City', how='left')

    # Fill missing values in the 'Population' column with the average population
    avg_population = 60000
    merged_df['Population'] = merged_df['Population'].fillna(avg_population)

    return merged_df

# Assuming 'shop_id' is the index of df_shops, you can call the function like this
df_shops_with_population = fill_population(df_shops, df_population)
df_shops_with_population

,shop_id,City,Population
0,0,Якутск,355443.0
1,1,Якутск,355443.0
2,2,Адыгея,60000.0
3,3,Балашиха,520962.0
4,4,Волжский,321479.0
5,5,Вологда,313944.0
6,6,Воронеж,1057681.0
7,7,Воронеж,1057681.0
8,8,Воронеж,1057681.0
9,9,Выездная,60000.0


In [55]:
df_shops_with_population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   shop_id     60 non-null     int64  
 1   City        60 non-null     object 
 2   Population  60 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.9+ KB


Теперь можно объединить наш датасет sales_train с датасетом категорий и датасетом магазинов в один и затем проверить на дубликаты.

In [56]:
sales_train_full = sales_train.merge(df_items, on="item_id", how="left")
sales_train_full

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,category
0,2013-01-02,0,59,22154,999.0,1.0,Кино
1,2013-01-03,0,25,2574,399.0,2.0,Музыка
2,2013-01-05,0,25,2574,399.0,1.0,Музыка
3,2013-01-07,0,25,2574,399.0,1.0,Музыка
4,2013-01-08,0,25,2574,399.0,2.0,Музыка
...,...,...,...,...,...,...,...
1224434,2015-10-10,33,25,7409,299.0,1.0,Музыка
1224435,2015-10-09,33,25,7460,299.0,1.0,Музыка
1224436,2015-10-14,33,25,7459,349.0,1.0,Музыка
1224437,2015-10-22,33,25,7440,299.0,1.0,Музыка


In [57]:
sales_train_full = sales_train_full.merge(df_shops_with_population, on="shop_id", how="left")
sales_train_full

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,category,City,Population
0,2013-01-02,0,59,22154,999.0,1.0,Кино,Ярославль,577279.0
1,2013-01-03,0,25,2574,399.0,2.0,Музыка,Москва,13010112.0
2,2013-01-05,0,25,2574,399.0,1.0,Музыка,Москва,13010112.0
3,2013-01-07,0,25,2574,399.0,1.0,Музыка,Москва,13010112.0
4,2013-01-08,0,25,2574,399.0,2.0,Музыка,Москва,13010112.0
...,...,...,...,...,...,...,...,...,...
1224434,2015-10-10,33,25,7409,299.0,1.0,Музыка,Москва,13010112.0
1224435,2015-10-09,33,25,7460,299.0,1.0,Музыка,Москва,13010112.0
1224436,2015-10-14,33,25,7459,349.0,1.0,Музыка,Москва,13010112.0
1224437,2015-10-22,33,25,7440,299.0,1.0,Музыка,Москва,13010112.0


Думаю, стоит выделить отдельным признаком сезон (как номер квартала), в который был продан товар и удалить даты - у нас есть номер недели, возможно, этого было бы достаточно.

In [62]:
sales_train_full['quarter'] = pd.PeriodIndex(sales_train_full.date, freq='Q')
# Возьмем последнюю цифру, обозначающую номер квартала (сезон)
sales_train_full['quarter'] = sales_train_full.quarter.astype(str).apply(lambda x: str(x[-1]))

In [61]:
sales_train_full

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,category,City,Population,quarter
0,2013-01-02,0,59,22154,999.0,1.0,Кино,Ярославль,577279.0,1
1,2013-01-03,0,25,2574,399.0,2.0,Музыка,Москва,13010112.0,1
2,2013-01-05,0,25,2574,399.0,1.0,Музыка,Москва,13010112.0,1
3,2013-01-07,0,25,2574,399.0,1.0,Музыка,Москва,13010112.0,1
4,2013-01-08,0,25,2574,399.0,2.0,Музыка,Москва,13010112.0,1
...,...,...,...,...,...,...,...,...,...,...
1224434,2015-10-10,33,25,7409,299.0,1.0,Музыка,Москва,13010112.0,4
1224435,2015-10-09,33,25,7460,299.0,1.0,Музыка,Москва,13010112.0,4
1224436,2015-10-14,33,25,7459,349.0,1.0,Музыка,Москва,13010112.0,4
1224437,2015-10-22,33,25,7440,299.0,1.0,Музыка,Москва,13010112.0,4


Перерь можно проветить на дубликаты. Дубликатами будем считать, если одна и та же позиция, в одном и том же магазине в одну и ту же дату и за одну и ту же цену возникла в датасете. Напишем функцию.

In [63]:
def drop_duplicate(data, subset):
    print('Before drop shape:', data.shape)
    before = data.shape[0]
    data.drop_duplicates(subset,keep='first', inplace=True)
    data.reset_index(drop=True, inplace=True)
    print('After drop shape:', data.shape)
    after = data.shape[0]
    print('Total Duplicate:', before-after)

In [65]:
subset =['date','shop_id','item_id','item_price']
drop_duplicate(sales_train_full,subset=subset)

Before drop shape: (1224439, 10)
After drop shape: (1224434, 10)
Total Duplicate: 5


# Посмотрим на выбросы в продажах, но в пределах каждой категории товаров
Возможно, придется в будущем умельчить категории для более точного разделения и выбросы будут уменьшены

In [97]:
def filter_outliers_by_category(df, method='quantiles', lower_quantile=0.01, upper_quantile=0.99, z_score_threshold=3, groupby_column='category'):
    if method == 'quantiles':
        outlier_mask = df.groupby(groupby_column)['item_cnt_day'].transform(
            lambda x: (x < x.quantile(lower_quantile)) | (x > x.quantile(upper_quantile))
        )
    elif method == 'z_score':
        outlier_mask = df.groupby(groupby_column)['item_cnt_day'].transform(
            lambda x: np.abs((x - x.mean()) / x.std()) > z_score_threshold
        )
    else:
        raise ValueError("Invalid method provided. Choose either 'quantiles' or 'z_score'.")

    filtered_df = df[~outlier_mask]
    return filtered_df

# Assuming your DataFrame is named df, you can call the function like this
filtered_df = filter_outliers_by_category(sales_train_full, method='quantiles')
print('Before drop shape:', sales_train_full.shape)
before = sales_train_full.shape[0]
print('After drop shape:', filtered_df.shape)
after = filtered_df.shape[0]
print('Total Duplicate:', before-after)
# Or if you prefer using z-scores:
# filtered_df = filter_outliers_by_category(df, method='z_score', z_score_threshold=3)

Before drop shape: (1224434, 10)
After drop shape: (1211734, 10)
Total Duplicate: 12700


Удалим ненужные колонки и датасет готов к следующему этапу  - ELT.

In [98]:
filtered_df = filtered_df.drop(labels = ['date','item_id', 'item_price'], axis = 1)

In [96]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1211734 entries, 0 to 1224433
Data columns (total 8 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   date_block_num  1211734 non-null  int64  
 1   shop_id         1211734 non-null  int64  
 2   item_price      1211734 non-null  float64
 3   item_cnt_day    1211734 non-null  float64
 4   category        1211734 non-null  object 
 5   City            1211734 non-null  object 
 6   Population      1211734 non-null  float64
 7   quarter         1211734 non-null  object 
dtypes: float64(3), int64(2), object(3)
memory usage: 83.2+ MB


ВОПРОСЫ:
1. Правильно ли я сделала вывод  о том, что нужно убрать из модели магазины, которых нет в тестовом наборе (по некоторым из них почти нет данных продаж)?
2. Правильно ли я сделала вывод  о том, что нужно убрать из модели товары (item_id), которых нет в тестовом наборе или лучше их оставить?
3. Я сомневаюсь, что нужно было удалять выбросы, так как я совсем не знаю причину их происходжения, может, это не ошибка, а особенность данных. Тем более, в первую очередь я хочу попробовать XGBoost (или CatBoost), который неплохо работает с выбросами, но и RNN тоже можно было попробовать. Как ты думаешь, стоит ли это делать?
4. Как ты думаешь, стоило ли заморачиваться по поводу новых фичей category	City, Population и quarter? Особенно с Population, для которого придется отдельно прописывать код для поиска значения при обработке input значений в модель?...
5. Может быть я что-то упустила?

Я работаю в Google collab (у меня есть подписка и я могу здесь на GPU обучать модели), но я бы хотела потом оформить все в виде скриптов и модулей (используя ООП), но только после того, как мы обсудим первые 2 этапа.